In [1]:
import pandas as pd
import numpy as np
import aqi
from fancyimpute import KNN
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
import matplotlib.pyplot as plt
import warnings
import logging

warnings.filterwarnings('ignore')

d:\AQI_forecasting\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = r"D:\AQI_forecasting\backend\data\bangkok-air-quality.csv"
df = pd.read_csv(data_path)
df

,date,pm25,pm10,o3,no2,so2,co
0,2025/1/1,116,50,15,2,,
1,2025/1/2,119,56,19,2,,
2,2025/1/3,120,58,24,1,,
3,2025/1/4,114,62,22,2,,
4,2025/1/5,119,68,22,2,,
...,...,...,...,...,...,...,...
4021,2015/3/15,,,23,13,3,9
4022,2014/11/30,,,40,36,4,11
4023,2013/12/31,,,68,33,3,7
4024,2014/1/26,,,54,53,6,18


In [6]:
def remove_whitespace_cols(df):
    df.columns = df.columns.str.strip()
    return df

def cleaning_data(df):
    
    df = remove_whitespace_cols(df)
    if 'date' not in df.columns:
        raise ValueError('date column not found')
    else:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.set_index('date')
        df = df.sort_index()
    try:
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    except ValueError:
        raise ValueError('Invalid data in columns')
    return df

data_1 = remove_whitespace_cols(df)
data_2 = cleaning_data(data_1)
data_2

,pm25,pm10,o3,no2,so2,co
date,,,,,,
2013-12-31,NaN,NaN,68.0,33.0,3.0,7.0
2014-01-01,NaN,84.0,90.0,48.0,5.0,16.0
2014-01-02,NaN,84.0,119.0,63.0,7.0,14.0
2014-01-03,NaN,103.0,95.0,57.0,7.0,13.0
2014-01-04,NaN,101.0,52.0,34.0,1.0,NaN
...,...,...,...,...,...,...
2025-01-11,84.0,57.0,18.0,3.0,NaN,NaN
2025-01-12,101.0,66.0,20.0,2.0,NaN,NaN
2025-01-13,129.0,73.0,21.0,2.0,NaN,NaN


In [ ]:
def calc_aqi(row:pd.Series):
    pollutants = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']
    aqi_values = []
    for pollutant in pollutants:
        if not np.isnan(row[pollutant]):
            aqi_values.append(aqi.to_aqi([(aqi.POLLUTANT_PM25, row[pollutant])]))
    if len(aqi_values) == 0:
        return np.nan
    return np.max(aqi_values)

def Extract_aqi(row:pd.Series):
    pollutants = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']
    
    
    
    
    
data_3 = data_2.apply(cal_aqi, axis=1)
data_3

date
2013-12-31   NaN
2014-01-01   NaN
2014-01-02   NaN
2014-01-03   NaN
2014-01-04   NaN
              ..
2025-01-11   NaN
2025-01-12   NaN
2025-01-13   NaN
2025-01-14   NaN
2025-01-15   NaN
Length: 4026, dtype: float64